In [1]:
from langchain_core.tools import tool
import yfinance as yf
from yahooquery import search
from deep_translator import GoogleTranslator

# 한글 문자 범위를 이용해 한글 포함 여부 확인
def contains_korean(text):
    for char in text:
        if '\uac00' <= char <= '\ud7a3' or '\u3131' <= char <= '\u318e':
            return True
    return False

def get_ticker(company_name):
    try:            
        # 한글 포함 여부 확인
        is_korean = contains_korean(company_name)
        if is_korean:
            # 회사명을 영어로 번역
            translated = GoogleTranslator(source='auto', target='en').translate(company_name)
            
            # 번역된 이름으로 검색
            results = search(translated)
        else:
            results = search(company_name)
            
        # KSC, NYSE, NASDAQ, AMEX, JPX, HKG 순서로 찾기
        for quote in results['quotes']:
            if quote['exchange'] == 'KSC': # 한국
                return quote['symbol']
            elif quote['exchange'] == 'NYQ': # NYSE
                return quote['symbol']
            elif quote['exchange'] == 'NMS': # NASDAQ
                return quote['symbol']
            elif quote['exchange'] == 'JPX': # 일본
                return quote['symbol']
            elif quote['exchange'] == 'HKG': # 홍콩
                return quote['symbol']
            else:
                continue
        
        # KSC, NYSE, NASDAQ, AMEX에 없으면 None 반환
        return None
    except Exception as e:
        print(f"Error translating or searching for {company_name}: {e}")
        return None

In [2]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
llm_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_claude = ChatAnthropic(model="claude-3-5-haiku-20241022", temperature=0)
llm = llm_openai

In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변, 파이썬 리스트 형식이어야 함."),
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [4]:
def find_peer(company: str) -> list[str]:
    prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결
    peer_list = chain.invoke({"question": f"{company}와 사업구조가 비슷하고, 같은 산업 혹은 섹터에 속한 경쟁사는?"
                              "(코스피, 뉴욕거래소 등 상장된 회사만 찾으세요. 반드시 회사명만 출력해주세요.)"})
    return peer_list

In [5]:
def find_peer_PERs_tool(company: str):
    """기업과 동종 업계의 Peer Group PER 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)['answer']
    if ticker is None:
        return None
    
    peer_pers = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm
            if trailingPERttm <0 :
                continue
            peer_pers[peer] = trailingPERttm
        else:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm*0.7 # 외국 주식의 경우 PER을 30% 할인
            if trailingPERttm <0 :
                continue
            peer_pers[peer] = trailingPERttm
    
    average_peer_per = sum(peer_pers.values()) / len(peer_pers)

    return {
        "Peer PERs": peer_pers,
        "Peer list": peer_list,
        "Average Peer PER": average_peer_per
    }

In [6]:
@tool
def find_peer_PBRs_tool(company: str):
    """기업과 동종 업계의 Peer Group PBR 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)['answer']
    if ticker is None:
        return None
    
    peer_pbrs = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            print(ticker)
            equity = ticker.quarterly_balance_sheet.loc['Stockholders Equity'][0]
            market_cap=ticker.info["marketCap"]
            PBR = market_cap/equity
            if PBR <0 :
                continue
            peer_pbrs[peer] = PBR
        else:
            ticker = yf.Ticker(ticker)
            print(ticker)
            equity = ticker.quarterly_balance_sheet.loc['Stockholders Equity'][0]
            market_cap=ticker.info["marketCap"]
            PBR = market_cap/equity*0.7 # 외국 주식의 경우 PBR을 30% 할인
            if PBR <0 :
                continue
            peer_pbrs[peer] = PBR
    
    average_peer_pbr = sum(peer_pbrs.values()) / len(peer_pbrs)

    return {
        "Peer PBRs": peer_pbrs,
        "Peer list": peer_list,
        "Average Peer PBR": average_peer_pbr
    }

In [7]:
find_peer("LG에너지솔루션")

{'answer': ['삼성SDI',
  'CATL',
  'Panasonic',
  'SK On',
  'BYD',
  'Tesla',
  'Northvolt']}

In [8]:
find_peer_PBRs_tool("LG에너지솔루션")

/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_56518/2970809802.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  find_peer_PBRs_tool("LG에너지솔루션")


yfinance.Ticker object <006400.KS>


/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_56518/3039342426.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  equity = ticker.quarterly_balance_sheet.loc['Stockholders Equity'][0]


{'Peer PBRs': {'삼성SDI': 0.8463371228030395},
 'Peer list': ['삼성SDI',
  'CATL',
  'Panasonic',
  'SK On',
  'BYD',
  'Tesla',
  'Northvolt'],
 'Average Peer PBR': 0.8463371228030395}